In [1]:
import os
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/DNA_classifier/research'

In [2]:
os.chdir("../")
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/DNA_classifier'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [4]:
from dnaseq.constants import *
from dnaseq.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_tranformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )
        
        return data_transformation_config

In [5]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from dnaseq import logger

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def transform_data(self):
        def getKmers(sequence, size=6):
            return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]
        
        data = pd.read_table(self.config.data_path)
        data['kmers'] = data.apply(lambda x: getKmers(x['sequence']), axis=1)
        data = data.drop('sequence', axis=1)
        
        X_data = list(data['kmers'])
        for item in range(len(X_data)):
            X_data[item] = ' '.join(X_data[item])
            
        y_data = data.iloc[:, 0].values
        
        data = pd.DataFrame({'sequence': X_data, 'class': y_data})
        
        return data
    
    def train_test_spliting(self, data):
        train, test = train_test_split(data)
        
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)
        
        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)
    

In [6]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_tranformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data = data_transformation.transform_data()
    data_transformation.train_test_spliting(data)

except Exception as e:
    raise e

[2024-03-31 15:57:46,971: INFO: common: created directory at: artifacts]
[2024-03-31 15:57:46,973: INFO: common: created directory at: artifacts/data_transformation]
[2024-03-31 15:57:49,084: INFO: 2098405633: Splited data into training and test sets]
[2024-03-31 15:57:49,085: INFO: 2098405633: (3285, 2)]
[2024-03-31 15:57:49,086: INFO: 2098405633: (1095, 2)]
